# 1. Intro to Feature Stores

Feature Stores are central hubs that (you guessed it) store features. In other words, feature store allows you to transform raw data into feature values, store those values, and use them for (re)training a model or for making predictions. This sounds like a classic database so far, right? But when you're working with lots of data, across lots of teams, new problems can surface which feature stores help address.

- Different teams might be using the same feature name to refer to something slightly different.
- Different teams might be computing a feature in a different and potentially incorrect way.
- Computed features required by a model might not be available immediately when required to make predictions in production.
- Features may be being computed differently and incorrectly between training and serving (deployment).
- Features may be being computed differently and incorrectly between development, testing and production.
- Different teams might not have access to the central, correct store of features.

### Example
You might have a huge dataset, and features that you engineered are becoming painfully slow to compute due to the great amount of data to be processed. For example, you might have a useful feature "Average purchase value" which, if you have millions of purchases, could take a while to compute this value. Each time you compute a feature you have to process all data; an alternative would be storing a running feature, so next time you add more data, you can access your feature store and grab the feature without processing the whole dataset.

<p><img src=images/Feature_Stores_API.png></p>

Feature stores make it easy to:
<details>
  <summary> 
    <b>1. Use features without needing to know and check how they are computed</b>
  </summary>

  Most of the features depend on a couple of operations because those are not very specific to the field. However, some features might depend on a team of engineers that has a deep knowledge on the field, and has created complicated formulas for computing them. <br><br>
  
</details>


<details>
  <summary> 
    <b> 2. Automate feature computation, backfills, and logging</b>
  </summary>

  When data is ingested, features need to be recomputed. Feature stores can automatically compute the new value. Feature stores can also fill in computed feature values in historical data entries. This is called "backfilling".<br><br>
</details>

<details>
  <summary> 
    <b> 3. Share and reuse features across teams</b>
  </summary>

  Features created by the engineering team can be used by the data scientist team. But the other way around is also possible. The features you create can be useful for the engineering team to check if the pipeline they are developing will be useful for the whole team.<br><br>
</details>

<details>
  <summary> 
    <b> 4. Achieve consistency between training and serving data</b>
  </summary>

Features have to be computed consistently between the training phase and after a model has been deployed. If it's not, your performance may be different. A common pitfall can be that during training, features were computed in a way that was not documented, and in production things look very different. E.gg. forgetting to normalise data<br><br>
</details>

<details>
  <summary> 
    <b> 5. Monitor the health of feature pipelines in production</b>
  </summary>

  After registering features in a feature store, you will be able to check if the pipeline they are used in is producing good results. By tracking the features feature stores make easier observing which feature works and which doesn't when linked to the outcomes,.<br><br>
</details>


# Features - Offline or Online

You know by now that a feature is an input variable. However, what is the difference between _Offline_ and _Online_ features?

- __Offline features__: These are static, historic features that don’t change much, and are processed in batch. Usually, offline features are calculated via frameworks such as Spark or by simply running SQL queries against a given database and then using a batch inference process. These features can be properties like patient age, a transaction location, or an IP address.<br><br>

- __Online features__: These features are dynamic and require a processing engine to calculate, sometimes in near-real time. They often need to be served in ultra-low latency. Any feature which is an average over some history, for example, is an online feature. As another example, calculating a z-score for real-time fraud detection. In this case, the pipeline is built by calculating the mean and the standard deviation over a sliding window in real time. These calculations are much more challenging, requiring fast computation as well as fast access to the data. The data can be stored in memory or in a very fast key-value database.

An even more complicated type of feature is engineered by using an ML process to generate the feature from a datapoint. An example of this might be to create a “contains a positive product review” feature by using NLP to identify reviews discussing a product in a positive manner.

# Feature Store Components

There are 5 main components of a modern feature store: Transformation, Storage, Serving, Monitoring, and feature Registry.

## Serving

Feature stores serve feature data to models. Those models require a consistent view of features across training and serving. The definitions of features used to train a model must exactly match the features provided in online serving. When they don’t match, training-serving skew is introduced which can cause catastrophic and hard-to-debug model performance problems. This can be caused by different teams calculating features in slightly different ways, or forgetting to normalise data amongst other reasons.

Feature stores abstract away the logic and processing used to generate a feature, providing users an easy and canonical way to access all features in a company consistently across all environments in which they’re needed.

When retrieving data offline (i.e. for training), feature values are commonly accessed through notebook-friendly feature store SDKs. They provide point-in-time correct views of the state of the world for each example used to train a model (a.k.a. “time-travel”).
For online serving, a feature store delivers a single vector of features at a time made up of the freshest feature values. Responses are served through a high-performance API backed by a low-latency database.

## Storage

Feature stores persist feature data to support retrieval through feature serving layers. They typically contain both an online and offline storage layer to support the requirements of different feature serving systems.

Offline storage layers are typically used to store months’ or years’ worth of feature data for training purposes. Offline feature store data is often stored in data warehouses or data lakes like S3, BigQuery, Snowflake, Redshift. Extending an existing data lake or data warehouse for offline feature storage is typically preferred to prevent data silos.

Online storage layers are used to persist feature values for low-latency lookup during inference. They typically only store the latest feature values for each entity, essentially modeling the current state of the world. Online stores are usually eventually consistent, and do not have strict consistency requirements for most ML use cases. They are usually implemented with key-value stores like DynamoDB, Redis, or Cassandra.

<p align=center><img src=images/Feature_Store_Storage.png width=500></p>

Feature stores use an entity-based data model where each feature value is associated with an entity (e.g. a user) and a timestamp. An entity-based data model provides minimal structure to support standardized feature management, fits naturally with common feature engineering workflows, and allows for simple retrieval queries in production.

## Transformation

Operational ML applications require regular processing of new data into feature values so models can make predictions using an up-to-date view of the world. Feature stores both manage and orchestrate data transformations that produce these values, as well as ingest values produced by external systems. Transformations managed by feature stores are configured by definitions in a common feature registry.

_Most teams getting started with feature stores already have existing data pipelines producing feature values. This makes it very important for feature stores to be gradually adoptable and have first class integrations with existing data platforms, allowing teams to immediately operationalize existing ETL pipelines for their ML use cases._

| Feature Type 	| Definition 	| Example 	|
|---	|---	|---	|
| Batch Transform 	| Transformations that are applied only to data at rest 	| User country, product category 	|
| Streaming Transform 	| Transformations that are applied to streaming sources 	| Number of clicks per vertical per user in last 30 minutes, Number of views per listing in past hour 	|
| On-demand transform 	| Transformations  that are used to produce features  based on data that is only available  at the time of the prediction. These features cannot be pre-computed. 	| Is the user currently in a supported location? Similarity score between listing and search query 	|

Models need access to fresh feature values for inference. Feature stores accomplish this by regularly recomputing features on an ongoing basis. Transformation jobs are orchestrated to ensure new data is processed and turned into fresh new feature values. These jobs are executed on data processing engines (e.g. Spark or Pandas) to which the feature store is connected. 

Model development introduces different transformation requirements. When iterating on a model, new features are often engineered to be used in training datasets that correspond to historical events (e.g. all purchases in the past 6 months). To support these use cases, feature stores make it easy to run “backfill jobs” that generate and persist historical values of a feature for training. Some feature stores automatically backfill newly registered features for preconfigured time ranges for registered training datasets.

Transformation code is reused across environments preventing training-serving skew and frees teams from having to rewrite code from one environment to the next.

An example of a Feature Store containing the already mentioned three components can be seen here:

<p align=center> <img src=images/Feature_Stores_TSS.png> </p>

In the image, Feature Management involves the Transformation part, and Access can be seen as Serving

## Monitoring

When something goes wrong in an ML system, it’s usually a data problem. Feature stores are uniquely positioned to detect and surface such issues. They can calculate metrics on the features they store and serve that describe correctness and quality. Feature stores monitor these metrics to provide a signal of the overall health of an ML application.

Feature data can be validated based on user defined schemas or other structural criteria. Data quality is tracked by monitoring for drift and training-serving skew. E.g. feature data served to models are compared to data on which the model was trained to detect inconsistencies that could degrade model performance.

When running production systems, it’s also important to monitor operational metrics. Feature stores track operational metrics relating to core functionality. E.g. metrics relating to feature storage (availability, capacity, utilization, staleness) or feature serving (throughput, latency, error rates). Other metrics describe the operations of important adjacent system components. For example, operational metrics for external data processing engines (e.g. job success rate, throughput, processing lag and rate).

Feature stores make these metrics available to existing monitoring infrastructure. This allows ML application health to be monitored and managed with existing observability tools in the production stack.

Having visibility into which features are used by which models, feature stores can automatically aggregate alerts and health metrics into views relevant to specific users, models, or consumers.

## Registry

A critical component in all feature stores is a centralized registry of standardized feature definitions and metadata. The registry acts as a single source of truth for information about a feature in an organization. 

The registry is a central interface for user interactions with the feature store. Teams use the registry as a common catalog to explore, develop, collaborate on, and publish new definitions within and across teams.

The definitions in the registry configure feature store system behavior. Automated jobs use the registry to schedule and configure data ingestion, transformation, and storage. It forms the basis of what data is stored in the feature store and how it is organized. Serving APIs use the registry for a consistent understanding of which feature values should be available, who should be able to access them, and how they should be served.

The registry allows for important metadata to be attached to feature definitions. This provides a route for tracking ownership, project or domain specific information, and a path to easily integrate with adjacent systems. This includes information about dependencies and versions which is used for lineage tracking. 

To help with common debugging, compliance, and auditing workflows, the registry acts as an immutable record of what’s available analytically and what’s actually running in production.

So far, we’ve looked at the core minimal components of a feature store. In practice, companies often have needs like compliance, governance, and security that require additional enterprise-focused capabilities. That will be the topic of a future blog post. 

# Feature Store Providers

Here we are enumerating a (non-exhaustive) list of commercial Feature Stores:

- __Feast__: Feast has grown in popularity during the last months, and currently is one of the most used Feature Stores providers. It can be easily used with Python, and the library will handle many file management intricacies. This is a great option if you already created the transformation Pipeline
- __Tecton__: Another very popular Feature Store provider is Tecton, which includes end-to-end pipelines for your features. This, as opposed to Feast, it supports transformations.
- __AWS__: Amazon doesn't actually provide a stand-alone Feature Store service. Instead, it integrates this service in its SageMaker service.
- __Hospworks__: It supports the full stack mentioned in this lesson, and it also includes a nice User Interface. Additionally, it can integrate more third-party services, so you can extend your pipeline easily.
- __Iguazio__: This Feature Store has its own platform, and doesn't rely on other platforms to store the features. However, that doesn't mean that you can't use third-party services!

You can get more information about each provider in this [link](https://mlops.community/learn/feature-store/), where you can easily compare them.

During this module we are going to focus on Feast, since it is easy to integrate, it has a large community, and it is a great way to start with Feature Stores.